In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [5]:
# Define transformations for data normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST dataset normalization values
])

# Load train and test datasets
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:23<00:00, 426750.10it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 54624.61it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:05<00:00, 306532.05it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 565801.27it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [6]:
class DigitRecognizer(nn.Module):
  def __init__(self):
    super(DigitRecognizer, self).__init__()
    # Define layers
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # Input channel=1 (grayscale)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(28 * 28 * 32, 128)  # Flatten output of conv2
    self.fc2 = nn.Linear(128, 10)  # 10 output classes (digits 0-9)

  def forward(self, x):
    # Apply convolutions, pooling, activation (ReLU)
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 28 * 28 * 32)  # Flatten
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define transformations (assuming grayscale MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

try:
  # Load train and test datasets
  train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
  test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

  # Create data loaders
  train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
  test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
except Exception as e:
  print(f"Error loading data: {e}")
  exit(1)

# Define model
class DigitRecognizer(nn.Module):
  def __init__(self):
    super(DigitRecognizer, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(28 * 28 * 32, 128)
    self.fc2 = nn.Linear(128, 10)  # 10 output classes

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 28 * 28 * 32)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# Define model, loss function, and optimizer
model = DigitRecognizer()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Move model to GPU if available

for epoch in range(num_epochs):
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)  # Move data to device
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss


NameError: name 'F' is not defined

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(data_loader):
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in data_loader:
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  return correct / total

# Calculate accuracy on validation set
validation_acc = calculate_accuracy(test_loader)
print(f"Validation Accuracy: {validation_acc:.4f}")


In [ ]:
# Test the model on the unseen test set
test_acc = calculate_accuracy(test_loader)
print(f"Test Accuracy: {test_acc:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Get a batch of test data
images, labels = next(iter(test_loader))

# Get predictions
outputs = model(images)
_, predicted = torch.max(outputs.data, 1)

# Show a few images with predicted and actual labels
rows, cols = 4, 4
fig, axs = plt.subplots(rows, cols, figsize
